## 5.3 延后初始化

也就是在定义网络的时候不初始化。

需要注意的是PyTorch的相关功能还在开发，不建议使用。

### 5.3.1 实例化网络

In [1]:
import torch
from torch import nn
net = nn.Sequential(
    nn.LazyLinear(256),
    nn.ReLU(),
    nn.Linear(256, 10),
    )
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


d:\Program Files\miniconda3\envs\torch\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。 因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。

In [2]:
[net[i].state_dict() for i in range(len(net))]

[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[-0.0142, -0.0166,  0.0380,  ..., -0.0622, -0.0384,  0.0194],
                       [-0.0093, -0.0193,  0.0537,  ..., -0.0442,  0.0360, -0.0049],
                       [ 0.0043, -0.0448, -0.0320,  ..., -0.0421,  0.0392,  0.0544],
                       ...,
                       [-0.0171,  0.0143, -0.0298,  ...,  0.0535, -0.0094, -0.0263],
                       [-0.0598,  0.0340,  0.0030,  ...,  0.0598, -0.0130,  0.0540],
                       [-0.0012,  0.0111, -0.0596,  ...,  0.0237,  0.0322,  0.0073]])),
              ('bias',
               tensor([ 0.0451,  0.0169,  0.0377, -0.0131, -0.0474, -0.0607,  0.0547, -0.0460,
                       -0.0554,  0.0531]))])]

现在让我们看看当我们试图通过initialize函数初始化参数时会发生什么。

In [3]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


### 5.3.2 小结

- 延后初始化使框架能够自动推断参数形状，使修改模型架构变得容易，避免了一些常见的错误。
- 我们可以通过模型传递数据，使框架最终初始化参数。